In [24]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import urllib.request
import xlwt
import math

In [25]:
book = xlwt.Workbook(encoding="utf-8")

In [26]:
sheet1 = book.add_sheet("Sheet 1")

In [27]:
sheet1.write(0, 0, "Index")
sheet1.write(0, 1, "Company Name")
sheet1.write(0, 2, "Industry")
sheet1.write(0, 3, "Sub Industry")
sheet1.write(0, 4, "Company Type")
sheet1.write(0, 5, "Level Of Office")
sheet1.write(0, 6, "Address")
sheet1.write(0, 7, "City")
sheet1.write(0, 8, "State")
sheet1.write(0, 9, "Email")
sheet1.write(0, 10, "Website")
sheet1.write(0, 11, "Location")
sheet1.write(0, 12, "Contact Name")
sheet1.write(0, 13, "Contact Mobile")
sheet1.write(0, 14, "Contact Phone")

In [28]:
lineNumber=1 

In [29]:
def scrapeCompanyPage(link):

    mainPageLink = "http://www.vcsdata.com/" + str(link) 

    #Since this page is not dynamically loaded unlike the main results page, this can be scraped straight from page source.
    #Hence, selenium and chromedriver are not needed here. a urllib.request is enough to access the content.

    page=urllib.request.urlopen(mainPageLink).read()
    soup = BeautifulSoup(page, "html.parser")

    contentListing = soup.find("div", {"class": "contentBoxListing"}).find("div", {"class": "contentlisting"})

    contentBox = contentListing.find("div", {"class": "innertitlepackinnerpage"})

    address = contentBox.find("div", {"class": "span7"}).find("span", {"itemprop": "address"}).text

    if (address.split('\r\n')[-1][0:2] == "E:"):
        email = address.split('\r\n')[-1]

        address = address.split('\r\n')[0: len(address.split('\r\n'))-2 ]

        addressString = ""

        for line in address:
            addressString += line
            
        #print("\nEmail: "+ email[2:])
        sheet1.write(lineNumber, 9, email[2:])
        #print("\nAddress: "+ addressString)
        sheet1.write(lineNumber, 6, addressString)

    else:
        address = address.split('\r\n')

        addressString = ""

        for line in address:
            addressString += line
            
        #print("\nEmail: ")
        sheet1.write(lineNumber, 9, "")
        #print("\nAddress: "+ addressString)
        sheet1.write(lineNumber, 6, addressString)
        
    contentListingContent = contentListing.find("div", {"class": "content"}).find("div", {"class": "detailbox"}).find("div", {"class": "content"})

    try:
        city = contentListingContent.find("div", {"class": "list"}).find("div", {"class": "span7"})
        sheet1.write(lineNumber, 7, city.text.lstrip())
    except:
        print("\nError in city on page: "+ str(currentPageNumber) + " at Line: " + str(lineNumber) + "\n\n\n")

    try:
        state = contentListingContent.find("div", {"class": "list1"}).find("div", {"class": "span7"})
        sheet1.write(lineNumber, 8, state.text.lstrip())
    except:
        print("\nError in state on page: "+ str(currentPageNumber) + " at Line: " + str(lineNumber) + "\n\n\n")

    try:
        website = contentListingContent.findAll("div", {"class": "list"})[1].find("div", {"class": "span7"})
        sheet1.write(lineNumber, 10, website.text.lstrip())
    except:
        print("\nError in website on page: "+ str(currentPageNumber) + " at Line: " + str(lineNumber) + "\n\n\n")
    #print ("\nCity is: " + city.text.lstrip())
    #print ("\nState is: " + state.text.lstrip())
    #print ("\nWebsite is: " + website.text.lstrip())
    

    rowMarginData = contentBox.findAll("div", {"class": "margintop10"})

    try:
        subInd = rowMarginData[0].find("span", {"class": "pull_right"}).text[13:]

        #print ("Sub Industry is: " + subInd)
        sheet1.write(lineNumber, 3, subInd)
    except:
        print("\nError in Sub Ind on page: "+ str(currentPageNumber) + " at Line: " + str(lineNumber) + "\n\n\n")



    contentArr = soup.find("div", {"class": "TabbedPanelsContentGroup"}).findAll("div", {"class":"TabbedPanelsContent"})


    #repeaters1 = contentArr[1].find("div", {"class": "content"}).findAll("div",{"class":"repeater"})

    #for rep1 in repeaters1:
    #    print(rep1)

    try:
        repeaters2 = contentArr[2].find("div", {"class": "detailbox"}).find("div", {"class":"content"}).findAll("div", {"class":"listRow"})

        try:
            name = repeaters2[0].find("div", {"class": "span8"})
            sheet1.write(lineNumber, 12, name.text)
        except:
            print("\nError in Contact Name on page: "+ str(currentPageNumber) + " at Line: " + str(lineNumber) + "\n\n\n")


        try:
            mobile = repeaters2[1].find("div", {"class": "span8"}).text.split(" -")[0]
            sheet1.write(lineNumber, 13, mobile)
        except:
            print("\nError in Mobile on page: "+ str(currentPageNumber) + " at Line: " + str(lineNumber) + "\n\n\n")

        try:
            phone = repeaters2[2].find("div", {"class": "span8"}).text.split(" -")[0]
            sheet1.write(lineNumber, 14, phone)
        except:
            print("\nError in Phone on page: "+ str(currentPageNumber) + " at Line: " + str(lineNumber) + "\n\n\n")
    except:
            print("\nError in Contact Details on page: "+ str(currentPageNumber) + " at Line: " + str(lineNumber) + "\n\n\n")
            try:
                content = soup.find("div", {"class": "contentBoxListing"}).find("div", {"class":"contentlisting"}).find("div", {"class":"content"})
                phone = content.findAll("div", {"class":"detailbox"})[1].find("div", {"class":"content"}).find("div", {"class":"listRow"}).find("div", {"class":"span8"})
                sheet1.write(lineNumber, 14, phone.text)
            except:
                print("\nAGAIN Error in Contact Details on page: "+ str(currentPageNumber) + " at Line: " + str(lineNumber) + "\n\n\n")
    

    #print("\n\nCONTACT DETAILS: ")

    #print ("\nName: " + name.text)
    #print ("\nMobile: " + mobile)
    #print ("\nPhone: " + phone)


In [30]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())


Checking for win32 chromedriver:2.45 in cache
Driver found in C:\Users\ASHISH\.wdm\chromedriver\2.45\win32/chromedriver.exe


In [31]:
pageLink = "https://www.vcsdata.com/logistics.html?category=Courier/%20Logistics/%20Packaging/%20Transportation"

pageLink += "&page="

In [32]:
driver.get(pageLink+ "1")

In [33]:
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "pagination")))
innerHTML = driver.execute_script("return document.body.innerHTML")
soup = BeautifulSoup(innerHTML, "lxml")

In [34]:
pagination = soup.find("div", {"class": "extrapadding" }).find("div", {"class": "pagination" })

In [35]:
if (pagination):
    allPagesLinks = pagination.findAll("a")

In [36]:
numberOfPages = allPagesLinks[len(allPagesLinks)-2].text   #The total number of pages in results.

In [37]:
print("\nNumber of pages: "+ numberOfPages + "\n\n\n")


Number of pages: 214





In [38]:
prettysoup = soup.find("div", {"class": "contentBoxListing" }).findAll("div", {"class": "contentBoxListing" })

In [39]:
currentPageNumber = 1

In [40]:
while(currentPageNumber <= int(numberOfPages)):

    try:

        #The current page is launched on the browser.
        driver.get(pageLink+ str(currentPageNumber))

        #Waits for the page to be rendered completely ("span75" is the class of one of the innermost divs on the results section).
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "span75")))

        innerHTML = driver.execute_script("return document.body.innerHTML")
        soup = BeautifulSoup(innerHTML, "html.parser")

        #Scraping begings. One by one, each of the results unit (usually 8 on a page on this website) are accessed and their data extracted.

        prettysoup = soup.find("div", {"class": "contentBoxListing" }).findAll("div", {"class": "contentBoxListing" })

        for x in prettysoup:
            sheet1.write(lineNumber, 0, lineNumber)
            try:
                contentlisting = x.find("div", {"class": "contentlisting"})
                if (contentlisting):
                    name = contentlisting.find("div", {"class": "innertitle"})
                    linkToCompany = contentlisting.find("a", {"class": "link"}, href=True)
                if (name):
                    #print(name.text)
                    sheet1.write(lineNumber, 1, name.text)
                if (linkToCompany):
                    scrapeCompanyPage(linkToCompany['href'])
                content = contentlisting.find("div", {"class": "content"})
                if (content):
                    if (content.find("div", {"class": "list"})):
                        list0 = content.findAll("div", {"class": "list"})

                        ind = list0[0].find("div", {"class": "span75"})
                        loc = list0[1].find("div", {"class": "span75"})
                        if (ind):
                            #print(ind.text)
                            sheet1.write(lineNumber, 2, ind.text)

                        if (loc):
                            #print(loc.text)
                            sheet1.write(lineNumber, 11, loc.text)

                        list1 = content.findAll("div", {"class": "list1"})

                        compType = list1[0].findAll("div", {"class": "span5"})[1]
                        officeLevel = list1[1].findAll("div", {"class": "span5"})[1]
                        if (compType):
                            #print(compType.text)
                            sheet1.write(lineNumber, 4, compType.text)

                        if (officeLevel):
                            #print(officeLevel.text)
                            sheet1.write(lineNumber, 5, officeLevel.text)
            except:
                print("\nError on page: "+ str(currentPageNumber) + " at Line: " + str(lineNumber) + "\n\n\n")

            lineNumber+=1   #Line Number is incremented after every result, so every result goes on a new line with new index.

        print("\n\nPage number: "+ str(currentPageNumber) + " DONE!")   #A notification on console once a page is done.
        currentPageNumber+=1    #The page number is incremented so the next time loop runs the next page is accessed.
        book.save("absolute/path/to/result.xls")   #Saves to a .xls file after every page.
    except:
        print("\nError on page: "+ str(currentPageNumber))  #Usually comes up for empty results divs on the dynamically loaded page source.

driver.quit()   #Closes the browser and shuts off the driver once all pages are done.
print("\n\nCOMPLETE!!!")    #Notification on console after successfully scraping all the pages.



Page number: 1 DONE!

Error on page: 2


Page number: 2 DONE!

Error on page: 3


Page number: 3 DONE!

Error on page: 4


Page number: 4 DONE!

Error on page: 5


Page number: 5 DONE!

Error on page: 6


Page number: 6 DONE!

Error on page: 7

Error in city on page: 7 at Line: 62





Page number: 7 DONE!

Error on page: 8


Page number: 8 DONE!

Error on page: 9

Error in city on page: 9 at Line: 79




Error in Contact Name on page: 9 at Line: 79




Error in Mobile on page: 9 at Line: 79




Error in Phone on page: 9 at Line: 79





Page number: 9 DONE!

Error on page: 10

Error in city on page: 10 at Line: 86





Page number: 10 DONE!

Error on page: 11


Page number: 11 DONE!

Error on page: 12


Page number: 12 DONE!

Error on page: 13


Page number: 13 DONE!

Error on page: 14

Error in city on page: 14 at Line: 125




Error in Contact Name on page: 14 at Line: 125




Error in Mobile on page: 14 at Line: 125




Error in Phone on page: 14 at Line: 125





Page number: 14


Error in Contact Details on page: 66 at Line: 590




Error in Contact Details on page: 66 at Line: 592




Error in Mobile on page: 66 at Line: 593




Error in Phone on page: 66 at Line: 593





Page number: 66 DONE!

Error on page: 67

Error in Phone on page: 67 at Line: 596




Error in Phone on page: 67 at Line: 602




Error in Phone on page: 67 at Line: 603





Page number: 67 DONE!

Error on page: 68

Error in Contact Details on page: 68 at Line: 604




Error in Phone on page: 68 at Line: 610




Error in Phone on page: 68 at Line: 611




Error in Phone on page: 68 at Line: 612





Page number: 68 DONE!

Error on page: 69

Error in Phone on page: 69 at Line: 613




Error in Contact Details on page: 69 at Line: 615




Error in Phone on page: 69 at Line: 616




Error in Mobile on page: 69 at Line: 617




Error in Phone on page: 69 at Line: 617




Error in Contact Details on page: 69 at Line: 619




Error in Mobile on page: 69 at Line: 620




Error in Phone on page: 6


Error in Contact Details on page: 91 at Line: 815




Error in Contact Details on page: 91 at Line: 817




Error in Contact Details on page: 91 at Line: 818




Error in Contact Details on page: 91 at Line: 819





Page number: 91 DONE!

Error on page: 92

Error in Contact Details on page: 92 at Line: 820




Error in Contact Details on page: 92 at Line: 821




Error in Contact Details on page: 92 at Line: 822




Error in Contact Details on page: 92 at Line: 823




Error in Contact Details on page: 92 at Line: 824




Error in Contact Details on page: 92 at Line: 826




Error in Contact Details on page: 92 at Line: 827




Error in Contact Details on page: 92 at Line: 828





Page number: 92 DONE!

Error on page: 93

Error in Contact Details on page: 93 at Line: 829




Error in Contact Details on page: 93 at Line: 830




Error in Contact Details on page: 93 at Line: 831




Error in Contact Details on page: 93 at Line: 832




Error in Contact Details on page: 93 at Line: 833


Error in Contact Details on page: 108 at Line: 972





Page number: 108 DONE!

Error on page: 109

Error in Contact Details on page: 109 at Line: 973




Error in Contact Details on page: 109 at Line: 974




Error in Contact Details on page: 109 at Line: 975




Error in Contact Details on page: 109 at Line: 976




Error in Contact Details on page: 109 at Line: 977




Error in Contact Details on page: 109 at Line: 979




Error in Contact Details on page: 109 at Line: 980




Error in Contact Details on page: 109 at Line: 981





Page number: 109 DONE!

Error on page: 110

Error in Contact Details on page: 110 at Line: 982




Error in Contact Details on page: 110 at Line: 983




Error in Contact Details on page: 110 at Line: 984




Error in Contact Details on page: 110 at Line: 985




Error in Contact Details on page: 110 at Line: 986




Error in Contact Details on page: 110 at Line: 988




Error in Contact Details on page: 110 at Line: 989




Error in Contact Details on p


Error in Contact Details on page: 125 at Line: 1123




Error in Contact Details on page: 125 at Line: 1124




Error in Contact Details on page: 125 at Line: 1125





Page number: 125 DONE!

Error on page: 126

Error in Contact Details on page: 126 at Line: 1126




Error in Contact Details on page: 126 at Line: 1127




Error in Contact Details on page: 126 at Line: 1128




Error in Contact Details on page: 126 at Line: 1129




Error in Contact Details on page: 126 at Line: 1130




Error in Contact Details on page: 126 at Line: 1132




Error in Contact Details on page: 126 at Line: 1133




Error in Contact Details on page: 126 at Line: 1134





Page number: 126 DONE!

Error on page: 127

Error in Contact Details on page: 127 at Line: 1135




Error in Contact Details on page: 127 at Line: 1136




Error in Contact Details on page: 127 at Line: 1137




Error in Contact Details on page: 127 at Line: 1138




Error in Contact Details on page: 127 at Line: 1139




Error in Cont


Error in Contact Details on page: 165 at Line: 1484




Error in Contact Details on page: 165 at Line: 1485





Page number: 165 DONE!

Error on page: 166

Error in Contact Details on page: 166 at Line: 1486




Error in Contact Details on page: 166 at Line: 1487




Error in Contact Details on page: 166 at Line: 1488




Error in Contact Details on page: 166 at Line: 1489




Error in Contact Details on page: 166 at Line: 1490




Error in Contact Details on page: 166 at Line: 1492




Error in Contact Details on page: 166 at Line: 1493




Error in Contact Details on page: 166 at Line: 1494





Page number: 166 DONE!

Error on page: 167

Error in Contact Details on page: 167 at Line: 1495




Error in Contact Details on page: 167 at Line: 1496




Error in Contact Details on page: 167 at Line: 1497




Error in Contact Details on page: 167 at Line: 1498




Error in Contact Details on page: 167 at Line: 1499




Error in Contact Details on page: 167 at Line: 1501




Error in Cont


Error in Contact Details on page: 182 at Line: 1633




Error in Contact Details on page: 182 at Line: 1634




Error in Contact Details on page: 182 at Line: 1636




Error in Contact Details on page: 182 at Line: 1637




Error in Contact Details on page: 182 at Line: 1638





Page number: 182 DONE!

Error on page: 183

Error in Contact Details on page: 183 at Line: 1639




Error in Contact Details on page: 183 at Line: 1640




Error in Contact Details on page: 183 at Line: 1641




Error in Contact Details on page: 183 at Line: 1642




Error in Contact Details on page: 183 at Line: 1643




Error in Contact Details on page: 183 at Line: 1645




Error in Contact Details on page: 183 at Line: 1646




Error in Contact Details on page: 183 at Line: 1647





Page number: 183 DONE!

Error on page: 184

Error in Contact Details on page: 184 at Line: 1648




Error in Contact Details on page: 184 at Line: 1649




Error in Contact Details on page: 184 at Line: 1650




Error in Cont


Error on page: 199

Error in Contact Details on page: 199 at Line: 1783




Error in Contact Details on page: 199 at Line: 1784




Error in Contact Details on page: 199 at Line: 1785




Error in Contact Details on page: 199 at Line: 1786




Error in Contact Details on page: 199 at Line: 1787




Error in Contact Details on page: 199 at Line: 1789




Error in Contact Details on page: 199 at Line: 1790




Error in Contact Details on page: 199 at Line: 1791





Page number: 199 DONE!

Error on page: 200

Error in Contact Details on page: 200 at Line: 1792




Error in Contact Details on page: 200 at Line: 1793




Error in Contact Details on page: 200 at Line: 1794




Error in Contact Details on page: 200 at Line: 1795




Error in Contact Details on page: 200 at Line: 1796




Error in Contact Details on page: 200 at Line: 1798




Error in Contact Details on page: 200 at Line: 1799




Error in Contact Details on page: 200 at Line: 1800





Page number: 200 DONE!

Error on page

In [41]:
book.save("Logistics.xls")